# Solutions to the code challenge
First Step is cleaning the data

In [1]:
import pandas as pd
import numpy as np

items = pd.read_csv("data/items.csv")
items.drop_duplicates(subset=['id'],keep="first")
items["item_name"]=items["item_name"].map(str.lower)
items["category"]=items["category"].map(str.lower)


# Fill empty price rows
if np.where(pd.isna(items["price"]))[0].size != 0:
    # Remove outliers before we fill any missing values
    num_rows = len(items.index)
    items= items[np.abs(items["price"] - items["price"].mean()) <= (3 * items["price"].std())]
    items= items[np.abs(items["item_count"] - items["item_count"].mean()) <= (3 * items["item_count"].std())]
    print("Dropped ", num_rows-len(items.index), " rows due to outliers in turnover and item_count")

    print("Empty price values detected, auto fill with mean")
    mean = items["price"].mean()
    count = items["price"].isna().sum()
    items["price"].fillna(items["price"].mean())
    print("Filled " + str(count) + " rows with mean value: "+ str(mean))

# Assumption; we can only sell items for at least 1 quantity
# drop rows with quantity <= 0
if np.where(items["item_count"] <= 0)[0].size != 0:
    num_rows = len(items.index)
    print(np.where(items["item_count"] <= 0)[0])
    items = items.drop(np.where(items["item_count"] <= 0)[0])
    print("Dropped ", num_rows - len(df.index), " rows due to out of bounds quantity")

# Fill rows with mean
if np.where(pd.isna(items["item_count"]))[0].size != 0:
    print("Selling an amount smaller than 0, or none")
    mean = items["item_count"].mean()
    count = items["item_count"].isin([0,-1]).sum() + items["item_count"].isna().sum()
    items["item_count"].fillna(items["item_count"].mean())
    print("Filled " + str(count) + " rows with mean value: " + str(mean))

After cleaning,the data is saved in a new file, so we always have access to the original data

In [2]:
items.to_csv("data/items_clean.csv",index=False)

### Cleaning tags.tsv
Since tags.tsv does not contain numerical data, the cleaning process is simpler.
Use lower() on all strings, so we dot need to worry about case sensitivity.

Task 1 : Show that for each item there exists a (maybe empty) set of tags.
This is taken care of in the make_list function and an assertion error is thrown if we do not receive an (empty) set.

In [3]:
def make_list(set_str):
    """
    Helper function to create a list ot of the set string
    """
    if set_str != "set()":
        assert "{" in set_str and "}" in set_str
        set_str = set_str.split("{")[1].split("}")[0].split(",")
        set_str = [x.split("'")[1].strip() for x in set_str]
    else:
        # replacing "set()" with np.nan makes pandas functions work
        set_str = np.nan
    return set_str


tags = pd.read_csv("data/tags.tsv",sep="\t",converters={"tags":make_list})
tags["item_name"] = tags["item_name"].map(str.lower)
tags["category"] = tags["category"].map(str.lower)
tags = tags.drop_duplicates(subset="id")

tags = tags.explode("tags")

## Tasks
### Task 1
#### Number 1: how many diffrent items are there ?
For tags.tsv

In [4]:
tags_ixc = tags[["item_name","category"]].value_counts()
print("And ",len(tags_ixc),"diffrent item combinations in tags.tsv")

And  9975 diffrent item combinations in tags.tsv


For item.csv

In [5]:
items_ixc = items[["item_name","category"]].value_counts()
print("There are ",len(items_ixc),"diffrent item combinations in item.csv")

There are  9999 diffrent item combinations in item.csv


Combined

In [6]:
#so we get the over all number of diffrent combinations
ixc = pd.concat([items[["item_name","category"]],tags[["item_name","category"]]])
# so we dont count duplicates
ixc = ixc.drop_duplicates()
print("Overall there are ", len(ixc)," diffrent combinations of items and category in both files." )

Overall there are  9999  diffrent combinations of items and category in both files.


#### Number 2: How many categories do you see?
For tags.tsv

In [7]:
tags_cat_num = tags["category"].value_counts()
print("There are ",len(tags_cat_num),"different categories in tags.tsv.")
print("The top 10 categories are:")
print(tags_cat_num.iloc[:10])

There are  6117 different categories in tags.tsv.
The top 10 categories are:
bier                     107
biere                     91
alkoholfreie getränke     87
spirituosen               86
longdrinks                66
cocktails                 63
wein                      62
afg                       56
vorspeisen                51
alkoholfrei               50
Name: category, dtype: int64


For item.csv

In [8]:
items_cat_num = items["category"].value_counts()
print("There are ",len(items_cat_num),"different categories in items.csv.")
print("The top 10 categories are:")
print(items_cat_num.iloc[:10])

There are  6129 different categories in items.csv.
The top 10 categories are:
bier                     101
biere                     87
spirituosen               78
alkoholfreie getränke     73
vorspeisen                51
wein                      51
afg                       49
cocktails                 48
speisen                   45
dessert                   43
Name: category, dtype: int64


Combined:

In [9]:
categorys = pd.concat([items["category"],tags["category"]])
categorys_num = categorys.value_counts()
print("Overall there are ",len(categorys_num),"different categories in both files.")
print("The top 10 categories are:")
print(categorys_num.iloc[:10])

Overall there are  6129 different categories in both files.
The top 10 categories are:
bier                     208
biere                    178
spirituosen              164
alkoholfreie getränke    160
wein                     113
cocktails                111
afg                      105
longdrinks               104
vorspeisen               102
alkoholfrei               90
Name: category, dtype: int64


#### Number3: How many tags do you see?

In [10]:
tags_nan = tags.tags.value_counts()
print("we have ",len(tags_nan)," tags and ", tags.tags.isna().sum(), " entries with no tag")
print("The top 10 tags are:")
print(tags_nan.iloc[:10])

we have  10  tags and  5932  entries with no tag
The top 10 tags are:
other_non_alcoholic_drinks    931
spirits                       920
carbonated_softdrink          912
hot_drinks                    800
wine                          711
beer                          406
sparkling_wine                205
beauty_venues                  50
misc_fermented_beverage        33
beauty_products                13
Name: tags, dtype: int64


### Task 2

In [11]:
names = items["item_name"]
exact = names.str.match(r'(jaegermeister.*|jägermeister.*)').sum()
rough = names[names.str.match(r'(.*jaeger.*|.*jäger.*)')].reset_index().drop(columns=['index'])
rough = len(rough.drop(np.where(rough['item_name'].str.match(r'(.*schnitzel.* | .*pfanne.*)'))[0]))

print("There are ",exact, " exact matches for Jägermeister. With an broader search there are ",
      rough," potential matches.")

There are  24  exact matches for Jägermeister. With an broader search there are  29  potential matches.


### Task 3
with SQL:

In [12]:
from pandasql import sqldf
# In case we want to se how much landed in the None category use the first one instead
q = """SELECT tags.tags AS tag, SUM(items.price*items.item_count) AS price FROM items,tags
        WHERE tags.item_name=items.item_name and tags.category=items.category
        GROUP BY tags.tags
        ORDER BY price DESC
    """

q = """SELECT tags.tags AS tag, SUM(items.price*items.item_count) AS price FROM items,tags
        WHERE tags.item_name=items.item_name and tags.category=items.category and tags.tags!="None"
        GROUP BY tags.tags
        ORDER BY price DESC
    """
print(sqldf(q,locals()))

                          tag      price
0                        beer  203135.45
1                  hot_drinks  181953.80
2                     spirits  154303.62
3                        wine  149226.65
4        carbonated_softdrink  131476.52
5  other_non_alcoholic_drinks  120128.59
6              sparkling_wine   40714.13
7               beauty_venues   11393.25
8     misc_fermented_beverage   11348.75
9             beauty_products    3179.75


With Python:

In [13]:
comb = items.merge(tags,how='left',left_on=['item_name','category'],right_on=['item_name','category'],
                   suffixes=['_items','_tags']).replace(np.nan, 'None')
tag_set = set(comb['tags'].tolist())
res = []
for entry in tag_set:
    sum_tag = sum(comb[comb['tags']==entry].item_count * comb[comb['tags']==entry].price)
    res.append((entry,sum_tag))
res.sort(key=lambda x:x[1])
for entry in res[::-1]:
    print(entry)

('None', 2532819.659999998)
('beer', 203135.45000000068)
('hot_drinks', 181953.80000000005)
('spirits', 154303.62)
('wine', 149226.64999999988)
('carbonated_softdrink', 131476.51999999996)
('other_non_alcoholic_drinks', 120128.58999999972)
('sparkling_wine', 40714.13)
('beauty_venues', 11393.249999999996)
('misc_fermented_beverage', 11348.750000000004)
('beauty_products', 3179.75)


### Task 4
Done via SQL, I choose to display the id's because it is cleaner than showing the item name.

In [14]:
#With None as tag
q = """ SELECT tmp.tags, Group_Concat(tmp.id) AS Ids
        FROM (select tags.tags AS tags ,df.id AS id, row_number() over
                (PARTITION BY tags.tags ORDER BY items.item_count DESC) AS seqnum FROM tags, items
                 WHERE items.item_name=tags.item_name AND items.category=tags.category) AS tmp
        WHERE seqnum <= 3
        GROUP BY tmp.tags
    """

#Without
q = """ SELECT tmp.tags, Group_Concat(tmp.id) AS Ids
        FROM (SELECT tags.tags AS tags ,items.id AS id, row_number() over
                (PARTITION BY tags.tags ORDER BY items.item_count DESC) AS seqnum FROM tags, items
                 WHERE items.item_name=tags.item_name AND items.category=tags.category AND tags.tags!='None') AS tmp
        WHERE seqnum <= 3
        GROUP BY tmp.tags
    """
print(sqldf(q,locals()))

                         tags                      Ids
0             beauty_products    699074,1175060,185432
1               beauty_venues  1126800,1477456,1773670
2                        beer     288146,293983,286608
3        carbonated_softdrink    1495692,998222,639739
4                  hot_drinks    1012359,344134,442910
5     misc_fermented_beverage     976070,42736,1289101
6  other_non_alcoholic_drinks   1332105,2011354,564290
7              sparkling_wine    1844862,141116,581208
8                     spirits     965110,694262,240812
9                        wine    52063,1799236,1732319


With Python:

In [15]:
# this is duplicate code but easier to read in a notebook
comb = items.merge(tags,how='left',left_on=['item_name','category'],right_on=['item_name','category'],
                   suffixes=['_items','_tags']).replace(np.nan, 'None')
tag_set = set(comb['tags'].tolist())

res = []
for entry in tag_set:
    vals = comb[comb['tags']==entry].sort_values(by='item_count',ascending=False)
    res.append((entry,vals.id_items.iloc[:3].tolist()))
for entry in res:
    print(entry)


('beer', [288146, 293983, 286608])
('misc_fermented_beverage', [976070, 42736, 1289101])
('beauty_venues', [1477456, 1126800, 1773670])
('carbonated_softdrink', [1495692, 998222, 639739])
('sparkling_wine', [1844862, 141116, 581208])
('hot_drinks', [1012359, 344134, 442910])
('None', [1192169, 79229, 1600388])
('spirits', [965110, 694262, 240812])
('beauty_products', [699074, 1175060, 185432])
('other_non_alcoholic_drinks', [1332105, 2011354, 564290])
('wine', [52063, 1799236, 1732319])
